# Feature preprocessing
- 尝试将diagnosis分开，用tfidf
- 特征衍生，加入动态累积平均值
- 增加div特征


In [21]:
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [22]:
data = pd.read_csv('../data/curated/combined_data.csv')
data

ID  PtGender  PtAge       Diagnosis  FYZ-IgM   甲流   乙流  分组汇总  \
0     23765694       0.0   89.0  乏力低钾血症贫血低钾血症贫血      NaN  2.0  NaN   2.0   
1     23765697       1.0   34.0              发热      NaN  NaN  3.0   3.0   
2     23769968       1.0   23.0            呼吸困难      1.0  NaN  NaN   1.0   
3     23773946       1.0   17.0              咳嗽      1.0  NaN  NaN   1.0   
4     23774757       1.0   12.0          急性扁桃体炎      1.0  NaN  NaN   1.0   
...        ...       ...    ...             ...      ...  ...  ...   ...   
7967  24026827       0.0   23.0              发热      NaN  NaN  NaN   NaN   
7968  24026921       1.0   29.0              发热      NaN  NaN  NaN   NaN   
7969  24026931       0.0   29.0              发热      NaN  NaN  NaN   NaN   
7970  24027035       0.0   34.0              发热      NaN  NaN  NaN   NaN   
7971  24027216       0.0   24.0              发热      NaN  NaN  NaN   NaN   

        WBC   RBC  ...  MONO%  EO%  BASO%   IG#  IG%    CRP  HFLC1  HFLC2  \
0      8.69  3.11  ...    3.9  6.0    0.2  0.07  0.8    NaN   0.01    0.2   
1      3.00  4.29  ...   11.8  1.0    0.3  0.00  0.0   0.95   0.00    0.1   
2     21.35  4.30  ...    5.1  0.6    0.1  0.02  0.1  41.00   0.01    0.0   
3     11.48  3.99  ...    4.9  0.3    0.3  0.00  0.0  35.00   0.00    0.0   
4      6.28  4.77  ...   10.1  0.1    0.0  0.00  0.0  21.91   0.02    0.3   
...     ...   ...  ...    ...  ...    ...   ...  ...    ...    ...    ...   
7967   5.52  5.05  ...   13.9  0.2    0.2  0.01  0.2   4.42   0.01    0.2   
7968   6.83  4.02  ...    9.7  0.7    0.3  0.02  0.3   6.48   0.00    0.0   
7969   5.59  5.12  ...   14.3  6.8    0.2  0.04  0.7  12.50   0.01    0.2   
7970   6.06  4.91  ...   16.5  0.2    0.3  0.01  0.2   1.60   0.01    0.2   
7971   6.67  4.97  ...   15.0  1.8    0.4  0.01  0.1   2.70   0.01    0.1   

      NRBC#  NRBC%  
0       0.0    0.0  
1       0.0    0.0  
2       0.0    0.0  
3       0.0    0.0  
4       0.0    0.0  
...     ...    ...  
7967    NaN    NaN  
7968    NaN    NaN  
7969    NaN    NaN  
7970    NaN    NaN  
7971    NaN    NaN  

[7972 rows x 39 columns]

In [23]:
print(data.columns)

Index(['ID', 'PtGender', 'PtAge', 'Diagnosis', 'FYZ-IgM', '甲流', '乙流', '分组汇总',
       'WBC', 'RBC', 'HGB', 'HCT', 'MCV', 'MCH', 'MCHC', 'PLT', 'RDW-SD',
       'RDW-CV', 'PDW', 'MPV', 'P-LCR', 'PCT', 'NEUT', 'LY', 'MONO', 'EO',
       'BASO', 'NEUT%', 'LYBL%', 'MONO%', 'EO%', 'BASO%', 'IG#', 'IG%', 'CRP',
       'HFLC1', 'HFLC2', 'NRBC#', 'NRBC%'],
      dtype='object')


In [24]:
# count the number of missing values in 分组汇总
data['分组汇总'].isnull().sum()
# print all the different values in Diagnosis column
data['Diagnosis'].unique()

array(['乏力低钾血症贫血低钾血症贫血', '发热', '呼吸困难', '咳嗽', '急性扁桃体炎', '上呼吸道感染',
       '上呼吸道感染感冒风热证风热证', '上呼吸道感染咳嗽风热犯肺证风热犯肺证',
       '支气管炎咳嗽风热犯肺证过敏性鼻炎[变应性鼻炎]风热犯肺证', '支气管炎咳嗽风热犯肺证风热犯肺证',
       '肺炎咳嗽风热犯肺证风热犯肺证', '腹痛发热', '肺炎', '腹痛腰痛', '气管炎', '支气管炎',
       '支气管肺炎肺炎喘嗽风热闭肺证风热闭肺证', '发热上呼吸道感染', '呼吸道感染',
       '重症肺炎I型呼吸衰竭重症肺炎间质性肺炎真菌性肺炎重症肺炎型糖尿病伴血糖控制', '支气管肺炎咳嗽风热犯肺证风热犯肺证',
       '支气管肺炎', '腹痛', '咳嗽孕周', '呼吸道感染细菌性感染', '发热咳嗽', '急性上呼吸道感染', '发热肺炎',
       '呕吐腹泻', '咳嗽腹痛', '咽痛咳嗽', '意识模糊急性脑血管病发热肺炎颅内感染发热肝功能不全肾功能不全低蛋白血症心功能',
       '发热咽痛', '冠状动脉粥样硬化性心脏病心房颤动肾功能不全急性左心衰竭肾功能不全', '发热肺炎痰热阻肺证痰热阻肺证',
       '心功能不全心功能不全少寐胃痞阴虚证少寐', '软组织感染型糖尿病伴血糖控制不佳泌尿道感染低蛋白血症贫血泌尿道感染脑萎缩大疱性类',
       '呼吸困难肺炎结肠肿瘤具有心脏起搏器贫血结肠肿瘤心房颤动高血压糖尿病',
       '型糖尿病性肾病型糖尿病性周围神经病型糖尿病性周围血管病变型糖尿病伴多个并发症型糖尿型糖尿病性周围血管病变',
       '呼吸道感染扁桃体炎', nan, '气管炎咳嗽风热犯肺证风热犯肺证', '心功能不全', '发热咳嗽咽痛咽痛', '咽痛',
       '咽痛呼吸道感染肺胃实热证肺胃实热证', '咳嗽发热',
       '贫血心力衰竭心功能不全结肠肿瘤冠状动脉粥样硬化性心脏病心功能不全具有心脏起搏器低蛋白血症', '肢体无力呕吐', '晕厥',
       '胸痛', '咳嗽呼吸道感染', '肾移植状态发热高血压糖尿病血液透析状态高血压', '疼痛发热', '呼吸道感染咽炎',
     

In [25]:
data['甲流'].fillna(0, inplace=True)
data.loc[data['甲流'] == 2, '甲流'] = 1
data['FYZ-IgM'].fillna(0, inplace=True)

In [26]:
data['Diagnosis'] = data['Diagnosis'].fillna('unknown').astype(str)
def chinese_tokenizer(text):
    return jieba.lcut(text)

# 1. 分词
data['diagnosis_tokenized'] = data['Diagnosis'].apply(chinese_tokenizer)
# delete repeated features in the same row
data['diagnosis_tokenized'] = data['diagnosis_tokenized'].apply(lambda x: list(set(x)))

# 2. TF-IDF
tfidf = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
features = tfidf.fit_transform(data['diagnosis_tokenized'])



In [27]:
data.dropna(subset=['PtGender'], inplace=True)

In [42]:
X = data.drop(columns=['甲流', 'FYZ-IgM', 'PtGender', '乙流', '分组汇总', 'Diagnosis', 'diagnosis_tokenized', 'ID'])  
# let y include 'FYZ-IgM', '甲流'
y = data[['FYZ-IgM', '甲流']]
diag = data[['Diagnosis', 'diagnosis_tokenized']]
X.fillna(X.mean(), inplace=True)


### 特征衍生（动态累积平均数）

In [43]:
X_average = pd.DataFrame()
# 打乱X的顺序
# X = X.sample(frac=1).reset_index(drop=True)

for column in X.columns:
    X_average['cumulative_mean_' + column] = X[column].cumsum() / (X.index + 1)
    # X_average['cumulative_mean_' + column] = X[column].expanding().mean()

In [44]:
X_average

cumulative_mean_PtAge  cumulative_mean_WBC  cumulative_mean_RBC  \
0                 89.000000             8.690000             3.110000   
1                 61.500000             5.845000             3.700000   
2                 48.666667            11.013333             3.900000   
3                 40.750000            11.130000             3.922500   
4                 35.000000            10.160000             4.092000   
...                     ...                  ...                  ...   
7967              27.391605             7.691973             4.667849   
7968              27.391807             7.691865             4.667768   
7969              27.392009             7.691601             4.667824   
7970              27.392838             7.691396             4.667855   
7971              27.392413             7.691268             4.667893   

      cumulative_mean_HGB  cumulative_mean_HCT  cumulative_mean_MCV  \
0               78.000000            23.600000            75.900000   
1              106.000000            31.750000            84.450000   
2              115.333333            34.366667            87.000000   
3              118.000000            34.950000            88.275000   
4              122.000000            36.180000            87.860000   
...                   ...                  ...                  ...   
7967           138.208333            41.310341            88.613855   
7968           138.206174            41.309575            88.613728   
7969           138.207528            41.310013            88.613588   
7970           138.208757            41.310601            88.614227   
7971           138.209985            41.310738            88.613811   

      cumulative_mean_MCH  cumulative_mean_MCHC  cumulative_mean_PLT  \
0               25.200000            331.000000           257.000000   
1               28.250000            333.500000           195.500000   
2               29.200000            335.000000           226.000000   
3               29.775000            337.000000           233.250000   
4               29.600000            336.800000           223.000000   
...                   ...                   ...                  ...   
7967            29.642997            334.126381           232.816140   
7968            29.643054            334.127620           232.817292   
7969            29.642986            334.127478           232.816688   
7970            29.643044            334.125957           232.821729   
7971            29.643063            334.127822           232.822629   

      cumulative_mean_RDW-SD  ...  cumulative_mean_MONO%  cumulative_mean_EO%  \
0                  57.700000  ...               3.900000             6.000000   
1                  51.900000  ...               7.850000             3.500000   
2                  48.600000  ...               6.933333             2.533333   
3                  47.025000  ...               6.425000             1.975000   
4                  45.820000  ...               7.160000             1.600000   
...                      ...  ...                    ...                  ...   
7967               41.984714  ...               9.657696             0.946524   
7968               41.984239  ...               9.657701             0.946493   
7969               41.984442  ...               9.658284             0.947227   
7970               41.984331  ...               9.659142             0.947133   
7971               41.983580  ...               9.659812             0.947240   

      cumulative_mean_BASO%  cumulative_mean_IG#  cumulative_mean_IG%  \
0                  0.200000             0.070000             0.800000   
1                  0.250000             0.035000             0.400000   
2                  0.200000             0.030000             0.300000   
3                  0.225000             0.022500             0.225000   
4                  0.180000             0.018000             0.180000   


In [31]:
X_average.to_csv('../data/curated/combined_data_average.csv', index=False)

### 增加div特征

In [32]:
# 为了避免除以零的错误，替换所有的0值
X = X.replace(0, 1e-8)

# 生成除法特征
n_features = X.shape[1]
for i in range(n_features):
    for j in range(i+1, n_features):
        col_name = f"{X.columns[i]}_div_{X.columns[j]}"
        X[col_name] = X.iloc[:, i] / X.iloc[:, j]


/var/folders/51/hrjbd3s95q7_fqgml710nj2r0000gn/T/ipykernel_83983/74248701.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col_name] = X.iloc[:, i] / X.iloc[:, j]
/var/folders/51/hrjbd3s95q7_fqgml710nj2r0000gn/T/ipykernel_83983/74248701.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col_name] = X.iloc[:, i] / X.iloc[:, j]
/var/folders/51/hrjbd3s95q7_fqgml710nj2r0000gn/T/ipykernel_83983/74248701.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [33]:
# combine X, y and diag
X = pd.concat([X, y, diag], axis=1)

In [34]:
X.to_csv('../data/curated/combined_data_div.csv', index=False)

### 测试所有的二阶交互项是否important

In [35]:
# poly = PolynomialFeatures(degree=2)
# X_poly = poly.fit_transform(X)

X_train_poly, X_test_poly, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_poly = scaler.fit_transform(X_train_poly)
X_test_poly = scaler.transform(X_test_poly)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_poly, y_train)

y_pred = model.predict(X_test_poly)
print(classification_report(y_test, y_pred))

ValueError: could not convert string to float: '咽痛'

In [ ]:
# print feature importance
feature_names = poly.get_feature_names()
feature_importance = model.coef_[0]
sorted_idx = np.argsort(feature_importance)
print(sorted_idx)


In [ ]:
# original_feature_names = X_train.columns 

# # 获取多项式特征的名称
# poly_feature_names = poly.get_feature_names_out(input_features=original_feature_names)



In [ ]:
data.loc[data['甲流'] == 1, 'PtAge'].describe()

In [ ]:
# top_n_idx = sorted_idx[:20]
# top_n_feature_names = np.array(poly_feature_names)[top_n_idx]
# print("Top 20 features:", top_n_feature_names)